In [1]:
#coding=utf-8
##############################################
#
# Author:       Shen Wenrui
# Date:         20180506
# Description:
#
##############################################

import sys
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import threading
import time
import re

In [2]:
def getEmailListLength(tempEmailBrowser, emailContent_xpath):
    # Step 3: Email list:
    emailElementList = tempEmailBrowser.find_elements_by_xpath(emailContent_xpath)
    return len(emailElementList)

def getNextUpdateSec(tempEmailBrowser, emailContent_xpath_base):
    # Step 4: Get next update sec:
    remainUpdateSec_xpath = emailContent_xpath_base + "//table[@id='email_table']//div[@id='tick']"
    remainUpdateSec = re.findall(r"Next update in: (.+?) sec",
                                 tempEmailBrowser.find_element_by_xpath(remainUpdateSec_xpath).text)
    remainUpdateSec = remainUpdateSec[0] if len(remainUpdateSec)!=0 else '0'
    return int(remainUpdateSec)

def getLatestEmailDat(tempEmailBrowser, emailContent_xpath):
    # Step 3: Get the latest Email's date:
    latestEmailDate_xpath = emailContent_xpath + "[%d]/td[@class='td4']" % 1
    latestEmailDate = tempEmailBrowser.find_element_by_xpath(latestEmailDate_xpath).text
    return latestEmailDate

def getLatestEmailAuthCode(tempEmailBrowser, emailContent_xpath):
    # Step 4: Get the latest Email's context:
    latestEmailContent_xpath = emailContent_xpath + "[%d]/td[@class='td3']/span[@class='email-excerpt']" % 1
    latestEmailContent = tempEmailBrowser.find_element_by_xpath(latestEmailContent_xpath).text
    # Step 5: Parse out the Auth Code:
    return latestEmailContent

def waitAuthCode(tempEmailBrowser, email_xpath_base):
    emailContent_xpath_base = email_xpath_base + "/div[@id='tabs-content']/div[@id='inbox']"
    emailContent_xpath = emailContent_xpath_base + "//table[@id='email_table']/tbody[@id='email_list']/tr"

    oriEmailListLength = getEmailListLength(tempEmailBrowser, emailContent_xpath)
    print(oriEmailListLength)

    # waitting for freshing the inbox.
    for i in range(2):
        remainUpdateSec = getNextUpdateSec(tempEmailBrowser, emailContent_xpath_base)
        print("Next update in: " + str(remainUpdateSec) + " sec")

        print("Start : %s" % time.ctime())
        time.sleep(remainUpdateSec + 3)
        print("End : %s" % time.ctime())

        emailListLength = getEmailListLength(tempEmailBrowser, emailContent_xpath)
        print(emailListLength)

        if(emailListLength > oriEmailListLength):
            authCode = getLatestEmailAuthCode(tempEmailBrowser, emailContent_xpath)
            print("authCode: " + authCode)
            oriEmailListLength = emailListLength
            return authCode

        latestEmailDate = getLatestEmailDat(tempEmailBrowser, emailContent_xpath)
        print(latestEmailDate)

    return None

In [7]:
def getTempEmailDominSelectXpath(email_xpath_base):
    emailAddr_xpath_base = email_xpath_base + "/div[@class='show_address']/div[@class='col2']"
    emailDomain_xpath_select = emailAddr_xpath_base + "//select[@id='gm-host-select']"
    return emailDomain_xpath_select

def getTempEmailAddrXpath(email_xpath_base):
    emailAddr_xpath_base = email_xpath_base + "/div[@class='show_address']/div[@class='col2']"
    emailAddr_xpath = emailAddr_xpath_base + "/span[@id='email-widget']"
    return emailAddr_xpath

def getTempEmailAddr(tempEmailBrowser, email_xpath_base):
    # Step 1: Select the Domain:
    emailDomain_xpath_select = getTempEmailDominSelectXpath(email_xpath_base)
    emailDomain_select = Select(tempEmailBrowser.find_element_by_xpath(emailDomain_xpath_select))
    # "guerrillamail.com", "sharklasers.com"
    emailDomain_value = "sharklasers.com"
    emailDomain_select.select_by_value(emailDomain_value)

    # Step 2: Get the Email address:
    emailAddr_xpath = getTempEmailAddrXpath(email_xpath_base)
    tempEmailAddr = tempEmailBrowser.find_element_by_xpath(emailAddr_xpath).text
    return tempEmailAddr

In [8]:
def guerrillamailListener():
    try:
        # Open the browser.
        print("Open the browser")
        tempEmailBrowser = webdriver.Chrome()

        # Open the netpage of guerrillamail.
        guerrillamailUrl = 'https://www.guerrillamail.com/inbox'
        print("Open:" + guerrillamailUrl)
        #tempEmailBrowser.implicitly_wait(20)
        tempEmailBrowser.get(guerrillamailUrl)
        
        email_xpath_base = "/html/body/div[@id='guerrilla_mail']/div[@class='main-panel']"
        
        try:
            # Explicitly wait.
            WebDriverWait(tempEmailBrowser, 20, 0.5).until(
                EC.presence_of_element_located((By.XPATH, getTempEmailAddrXpath(email_xpath_base)))
            )
        except TimeoutException as err:
            print("Failed to load: " + guerrillamailUrl)
            return False
            
        print("Successfully load: " + guerrillamailUrl)

        
        tempEmailAddr = getTempEmailAddr(tempEmailBrowser, email_xpath_base)
        print("Get temp Email Addr: " + tempEmailAddr)



        # Open guerrillamail & get email addr.


        # Looply blocking wait for signal that auth code has been sent.
    
        return True
    
    except Exception as err:
        print("[ERROR]:" + str(err))
        
    finally:
        # Close the browser.
        tempEmailBrowser.quit()
        print("Close the browser.")

In [9]:
class guerrillamailThread (threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print ("开启线程： " + self.name)
        print ("Start the Main process: applying temp email.")
        
        guerrillamailListener()
        
        print ("结束线程： " + self.name)
        print ("End the Main process.")
            
        # 获取锁，用于线程同步
        #threadLock.acquire()
        #print_time(self.name, self.counter, 3)
        # 释放锁，开启下一个线程
        #threadLock.release()

In [10]:
# 创建新线程
thread1 = guerrillamailThread(1, "Thread-1", 1)

# 开启新线程
thread1.start()

开启线程： Thread-1
Start the Main process: applying temp email.
Open the browser
Open:https://www.guerrillamail.com/inbox
Successfully load: https://www.guerrillamail.com/inbox
Get temp Email Addr: chn50m+925tzj0l9vlqc@sharklasers.com
Close the browser.
结束线程： Thread-1
End the Main process.


In [ ]:
authCode = waitAuthCode(tempEmailBrowser, email_xpath_base)
if authCode is None:
    print("Failed")


In [ ]:


class myThread (threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print ("开启线程： " + self.name)
        # 获取锁，用于线程同步
        threadLock.acquire()
        print_time(self.name, self.counter, 3)
        # 释放锁，开启下一个线程
        threadLock.release()

def print_time(threadName, delay, counter):
    while counter:
        time.sleep(delay)
        print ("%s: %s" % (threadName, time.ctime(time.time())))
        counter -= 1

threadLock = threading.Lock()
threads = []

# 创建新线程
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# 开启新线程
thread1.start()
thread2.start()

# 添加线程到线程列表
threads.append(thread1)
threads.append(thread2)

# 等待所有线程完成
for t in threads:
    t.join()
print ("退出主线程")